In [1]:
# !pip install efficientnet-pytorch sklearn

In [2]:
import os
import random
import time

import numpy as np
import pandas as pd
import torch
import torchvision
from PIL import Image
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

import models
from dataset_generator import DatasetGenerator

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_size = 224

class_to_idx = {
    'normal': 0,
    'pneumonia': 1,
    'COVID-19': 2
}

In [3]:
print(torch.__version__)
print(torchvision.__version__)

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

1.4.0
0.5.0


In [4]:
image_dir = 'data'
train_csv_file = 'newtrain_split.txt'
test_csv_file = 'test_split.txt'
valid_csv_file = 'valid_split.txt'

In [5]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([
    transforms.RandomOrder([
        transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15, resample=Image.BILINEAR),
        transforms.RandomResizedCrop(image_size, scale=(0.9, 1.0)),
    ]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

valid_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

test_transforms = valid_transforms 

In [6]:
# TODO: Load the datasets with DatasetGenerator
train_dir = os.path.join(image_dir, 'train')
train_dataset = DatasetGenerator(train_csv_file, train_dir, transform=train_transforms)
image, label = next(iter(train_dataset))
print(label)
display(image)

pneumonia


tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -1

In [7]:
# TODO: Load the datasets with DatasetGenerator
test_dir = os.path.join(image_dir, 'test')
test_dataset = DatasetGenerator(test_csv_file, test_dir, transform=test_transforms)
image, label = next(iter(test_dataset))
print(label)
display(image)

pneumonia


tensor([[[-0.4226, -0.4397, -0.4397,  ...,  0.9988,  0.8618,  0.6906],
         [-0.3369, -0.3369, -0.3369,  ...,  0.5022,  0.4166,  0.3481],
         [-0.2684, -0.2513, -0.2856,  ...,  0.3994,  0.3994,  0.4166],
         ...,
         [ 1.8893,  1.9064,  1.9578,  ...,  2.1119,  2.1975,  2.1804],
         [ 1.9235,  1.9407,  1.9407,  ...,  2.0948,  2.1804,  2.1804],
         [ 1.9578,  1.9749,  1.9578,  ...,  2.1119,  2.1633,  2.1633]],

        [[-0.3025, -0.3200, -0.3200,  ...,  1.1506,  1.0105,  0.8354],
         [-0.2150, -0.2150, -0.2150,  ...,  0.6429,  0.5553,  0.4853],
         [-0.1450, -0.1275, -0.1625,  ...,  0.5378,  0.5378,  0.5553],
         ...,
         [ 2.0609,  2.0784,  2.1310,  ...,  2.2885,  2.3761,  2.3585],
         [ 2.0959,  2.1134,  2.1134,  ...,  2.2710,  2.3585,  2.3585],
         [ 2.1310,  2.1485,  2.1310,  ...,  2.2885,  2.3410,  2.3410]],

        [[-0.0790, -0.0964, -0.0964,  ...,  1.3677,  1.2282,  1.0539],
         [ 0.0082,  0.0082,  0.0082,  ...,  0

In [8]:
#TODO: Load the datasets with DatasetGenerator
valid_dir = os.path.join(image_dir,'train')
valid_dataset = DatasetGenerator(valid_csv_file, valid_dir, transform=valid_transforms)
image,label = next(iter(valid_dataset))
print(label)
display(image)

pneumonia


tensor([[[-2.1179, -2.1179, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.0837,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.0665,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0182,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0007,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -1.9832,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.7870,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.7696,  ..., -1

In [9]:
# TODO: Build and train your network
def load_pretrained_model(arch):
    model_func = getattr(models, arch)
    model = model_func()
    model.arch = arch
    
    return model

In [10]:
# convert labels to tensor
def labels_to_tensor(labels):
    label_indices = [class_to_idx[label] for label in labels]
    label_indices = np.array(label_indices, int)
    label_indices = torch.LongTensor(label_indices)
    return label_indices

labels_to_tensor(['normal', 'pneumonia', 'COVID-19'])

tensor([0, 1, 2])

In [11]:
def validate(model, valid_dataloader, loss_func, device):
    #track accuracy and loss 
    accuracy = 0
    test_loss = 0
    
    with torch.no_grad(): #deactivates requires_grad flag, disables tracking of gradients 
        for images, labels in valid_dataloader: #iterate over images and labels in valid dataset
            labels = labels_to_tensor(labels)
            images, labels = images.to(device), labels.to(device) #move a tensor to a device
            log_ps = model.forward(images) #log form of probabilities for each label
            test_loss += loss_func(log_ps, labels).item() #.item() returns loss value as float, compare prob to actual 
            
            ps = torch.exp(log_ps) #gets rid of log 
            equality = (labels.data == ps.max(dim=1)[1]) #takes highest probability
            accuracy += torch.mean(equality.type(torch.FloatTensor))

    return test_loss, accuracy

In [12]:
# TODO: Do validation on the test set
def test_model(model, test_loader, device):
    #track accuracy, move to device, switch on eval mode
    accuracy = 0
    model.to(device)
    model.eval()
    
    with torch.no_grad():
        for images, labels in iter(test_loader):
            labels = labels_to_tensor(labels)
            images, labels = images.to(device), labels.to(device) #move a tensor to a device
            log_ps = model.forward(images)
            ps = torch.exp(log_ps)
            
            equality = (labels.data == ps.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
        model_accuracy = accuracy/len(test_loader)
        model.accuracy = model_accuracy
    
    return model.accuracy           

In [13]:
# TODO: Save the checkpoint
def save_checkpoint(checkpoint_path, model):
    checkpoint = {
        #TODO: Save the model arch, accuracy, classifier, class_to_idx
        'arch':model.arch,
        'accuracy':model.accuracy,
        'class_to_idx':class_to_idx,
        'state_dict':model.state_dict()
    }
    torch.save(checkpoint, checkpoint_path)
    print('Saved the trained model: %s' % checkpoint_path)

In [14]:
# TODO: Using the image datasets and the trainforms, define the dataloaders
batch_size = 64
dataloaders = {"train": DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True),
               "test": DataLoader(test_dataset, batch_size=batch_size, num_workers=16, shuffle=True),
              "valid":DataLoader(valid_dataset, batch_size=batch_size, num_workers=16, shuffle=True)}

In [15]:
device = 'cpu'
cuda = torch.cuda.is_available()
if cuda:
    device = 'cuda'
device

'cuda'

In [16]:
available_models = [m for m in dir(models) if 'Net' in m and 'Model' not in m]
available_models

['DenseNet121', 'DenseNet169', 'EfficientNet5', 'ResNet50']

In [17]:
arch = 'EfficientNet5'
pretrained_model = load_pretrained_model(arch)
optimizer = optim.Adam(pretrained_model.get_optimizer_parameters(),
                       lr=0.00001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
criterion = nn.NLLLoss()

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /home/ubuntu/.cache/torch/checkpoints/efficientnet-b5-b6417697.pth



Loaded pretrained weights for efficientnet-b5


In [18]:
def train(model, loss_func, optimizer, dataloaders, device, epochs, checkpoint_prefix, print_every=20):
    device = torch.device(device)
    model.to(device)

    train_loader = dataloaders['train']
    valid_loader = dataloaders['valid']

    epoch_start = time.time()
    max_acc = 0.0

    # loop to train for number of epochs
    for e in range(epochs):
        running_loss = 0
        batch_start = time.time()
        steps = 0

        for images, labels in train_loader:
            # within each loop, iterate train_loader, and print loss
            label_indices = labels_to_tensor(labels)
            images, labels = images.to(device), label_indices.to(device)
            optimizer.zero_grad()
            log_ps = model.forward(images)
            loss = loss_func(log_ps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            steps += 1

            if steps % print_every == 0:
                model.eval()
                valid_loss, valid_accuracy = validate(model, valid_loader, loss_func, device)
                model.train()
                batch_time = time.time() - batch_start
                print(
                    "Epoch: {}/{}..".format(e + 1, epochs),
                    "Step: {}..".format(steps),
                    "Training Loss: {:.3f}..".format(running_loss / len(train_loader)),
                    "Test Loss: {:.3f}..".format(valid_loss / len(valid_loader)),
                    "Test Accuracy: {:.3f}..".format(valid_accuracy / len(valid_loader)),
                    "Batch Time: {:.3f}, avg: {:.3f}".format(batch_time, batch_time / steps)
                )

        model.eval()
        valid_loss, valid_accuracy = validate(model, valid_loader, loss_func, device)
        model.train()
        epoch_time = time.time() - epoch_start

        if valid_accuracy > max_acc:
            max_acc = valid_accuracy
            model.accuracy = valid_accuracy
            save_checkpoint('%s.pth.tar' % checkpoint_prefix, model)
            print ('Epoch [{}] [save] Accuracy={:.3f} time: {:.3f}, avg: {:.3f}'
                   .format(e + 1, valid_accuracy / len(valid_loader), epoch_time, epoch_time / (e + 1)))
        else:
            print ('Epoch [{}] [----] Accuracy={:.3f} time: {:.3f}, avg: {:.3f}'
                   .format(e + 1, valid_accuracy / len(valid_loader), epoch_time, epoch_time / (e + 1)))

    return model

In [19]:
#device = 'cpu'
epochs = 2
training_start = time.time()
checkpoint_prefix = os.path.join('checkpoints','%s-%d'%(arch, training_start))
    
trained_model = train(pretrained_model, criterion, optimizer, dataloaders, device, epochs, checkpoint_prefix, print_every=20)
print('%.2f seconds taken for model training' % (time.time() - training_start))

Epoch: 1/2.. Step: 20.. Training Loss: 0.111.. Test Loss: 1.050.. Test Accuracy: 0.594.. Batch Time: 93.928, avg: 4.696
Epoch: 1/2.. Step: 40.. Training Loss: 0.219.. Test Loss: 1.030.. Test Accuracy: 0.596.. Batch Time: 155.609, avg: 3.890
Epoch: 1/2.. Step: 60.. Training Loss: 0.324.. Test Loss: 1.002.. Test Accuracy: 0.635.. Batch Time: 218.787, avg: 3.646
Epoch: 1/2.. Step: 80.. Training Loss: 0.427.. Test Loss: 0.984.. Test Accuracy: 0.629.. Batch Time: 282.833, avg: 3.535
Epoch: 1/2.. Step: 100.. Training Loss: 0.527.. Test Loss: 0.962.. Test Accuracy: 0.689.. Batch Time: 347.452, avg: 3.475
Epoch: 1/2.. Step: 120.. Training Loss: 0.625.. Test Loss: 0.944.. Test Accuracy: 0.728.. Batch Time: 411.659, avg: 3.430
Epoch: 1/2.. Step: 140.. Training Loss: 0.721.. Test Loss: 0.925.. Test Accuracy: 0.785.. Batch Time: 475.467, avg: 3.396
Epoch: 1/2.. Step: 160.. Training Loss: 0.816.. Test Loss: 0.921.. Test Accuracy: 0.797.. Batch Time: 538.951, avg: 3.368
Epoch: 1/2.. Step: 180.. Trai

In [20]:
test_loader = dataloaders['test']
test_accuracy = test_model(trained_model, test_loader, device)
print(test_accuracy)

tensor(0.7373)
